In [12]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [14]:
model = torch.load('./model/D_B.pt').eval()

In [8]:

label = en(['Mr President, Commissioner, I would like to thank Mrs Schroedter for an excellent report.', 'She has gone into the issue in some depth and in the committee debate she took account of many of the amendments that have been tabled regarding this report.'])
pred = en(['Mr President, ladies and gentlemen, I thank Mrs Schroedter for the sound report.', "You have dealt with the issue in detail and, during the committee's discussion, you have taken account of many of the amendments that have been tabled to this report."])

In [9]:
#real sentences
x = label[0].cuda()
#delte 0 give huge diff
       
# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_label =  model.classifier(distr)#(bs,1)
print(ret_label)

print(model(x,x_attn))

tensor([[ 1363,  1661,     6, 14595,     6,    27,   133,   114,    12,  2763,
          8667,   180, 10363,    15,    26,   449,    21,    46,  1287,   934,
             5,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  451,    65,  2767,   139,     8,   962,    16,   128,  4963,    11,
            16,     8,  4492,  5054,   255,   808,   905,    13,   186,    13,
             8, 12123,     7,    24,    43,   118,   953,    26,  1918,    48,
           934,     5,     1]], device='cuda:0')
torch.Size([2, 33])
		---------embedding
torch.Size([2, 33, 512])
tensor([[[-15.8958,   0.3818,   3.2577,  ...,  -4.6827,  27.4352,  17.6700],
         [ -1.9777,  -6.5448,  28.4217,  ..., -56.8084, -72.0019,   8.4903],
         [ 12.6347,   7.9227, -11.0104,  ...,   7.9381,  -6.7786,  -3.4063],
         ...,
         [ -1.9719,   0.1815,  -7.0937,  ...,  -0.4849,   2.7084,  -2.8707],
         [ -1.9719,   0.1815,  -7.0937,  ...,  -0.4849,   

In [10]:
#real sentences
x = pred[0].cuda()
       
# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
print(torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_pred =  model.classifier(distr)#(bs,1)
print(ret_pred)

torch.Size([2, 37])
		---------embedding
torch.Size([2, 37, 512])
tensor([[[-15.8958,   0.3818,   3.2577,  ...,  -4.6827,  27.4352,  17.6700],
         [ -1.9777,  -6.5448,  28.4217,  ..., -56.8084, -72.0019,   8.4903],
         [ 12.6347,   7.9227, -11.0104,  ...,   7.9381,  -6.7786,  -3.4063],
         ...,
         [ -1.9719,   0.1815,  -7.0937,  ...,  -0.4849,   2.7084,  -2.8707],
         [ -1.9719,   0.1815,  -7.0937,  ...,  -0.4849,   2.7084,  -2.8707],
         [ -1.9719,   0.1815,  -7.0937,  ...,  -0.4849,   2.7084,  -2.8707]],

        [[ 16.2764,  -0.3113, -10.2170,  ...,  -3.1309,  -9.5889,  33.4791],
         [ 16.4463, -13.6224, -23.6603,  ...,   8.4848,   0.2268,  -0.3471],
         [-20.3548, -31.3420, -30.7264,  ...,   8.8942,   4.9634,  36.2160],
         ...,
         [  8.8538,   7.8113,   5.6407,  ...,  16.5174, -43.0303,   1.5869],
         [ -4.3337,   7.3677, -14.5860,  ...,  13.0438,  -2.7213,  -3.1239],
         [ 12.6469,   8.1988, -11.5319,  ...,   7.8185,  

In [11]:
criterionGAN = torch.nn.MSELoss()
pred_real = ret_label
loss_D_real = criterionGAN(pred_real, torch.ones((pred_real.shape[0],1),device='cuda'))
print(loss_D_real)
# Fake
pred_fake = ret_pred
loss_D_fake = criterionGAN(pred_fake, torch.zeros((pred_fake.shape[0],1),device='cuda'))
# Combined loss and calculate gradients
print(loss_D_fake)


loss_D = (loss_D_real + loss_D_fake) * 0.5
print(loss_D)

tensor(0.7821, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0142, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3981, device='cuda:0', grad_fn=<MulBackward0>)


,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier

## FAKE dont end with 1, cuz i use model.generate but not self.generate